In [4]:
# -*- coding: utf-8 -*-
# @Time    : 2022/3/30 20:43
# @Author  : huangkai
# @File    : search_content.py


import sys

sys.path.append('../recall/nlp4es8/ir/')
sys.path.append('../utils')

from es_config import Config


# from utils.logger_config import base_logger
# from utils.args import FLAGS
# from ir.index_hnsw import Index


class Search(object):
    def __init__(self, config, index_name):
        # base_logger.info("Searching ...")
        self.config = config
        self.es = self.config.es

        self.index_name = index_name

    def searchAnswer(self, question, ):
        body = {
            "query": {
                "multi_match": {
                    "query": question,
                    "fields": ["idx_document"],  # 在question字段中匹配查询
                    "type": "most_fields",
                }
            }
        }

        # es相关配置

        res = self.es.search(index=self.index_name, body=body, request_timeout=30, size=self.config.top_n)

        topn = res['hits']['hits']

        result = []
        for data in topn:
            result.append(
                (
                    data['_source']['index'],
                    data['_source']['idx_document'],
                    data['_source']['ori_document']
                )
            )
        return result


In [5]:
import pandas as pd
import json

index_file_content_config = Config()
index_file_content_config.index_name = "kbqa"
index_file_content_config.top_n=1
search_name = Search(index_file_content_config, "kbqa")



In [4]:
while True:
    query = input("please input")
    result = search_name.searchAnswer(query)
    print(result[0])
    for data in result:
        print("question:%s  question_id:%s  " % (data[0], data[1]))

please input足球运动员莫侯斯所在的城市是哪个
('6068036', '莫侯斯 ||| 运动项目 ||| 足球')
question:6068036  question_id:莫侯斯 ||| 运动项目 ||| 足球  
question:6068038  question_id:莫侯斯 ||| 所在城市 ||| 米盧斯  
question:7483668  question_id:侯斯特 ||| 职业 ||| 运动员 足球  
question:26744282  question_id:拉莫斯(洪都拉斯足球运动员) ||| 运动项目 ||| 足球  
question:1070692  question_id:莫里斯(足球运动员) ||| 别名 ||| 莫里斯  
question:32913968  question_id:斯莫德斯 ||| 职业 ||| 运动员 足球  
question:10205418  question_id:纳塞夫·莫里斯(足球运动员) ||| 运动项目 ||| 足球  
question:8946580  question_id:霍莫斯 ||| 职业 ||| 运动员 足球  
question:9088935  question_id:迪莫斯 ||| 职业 ||| 运动员 足球  
question:40998095  question_id:夏莫斯 ||| 职业 ||| 运动员 足球  
question:19996452  question_id:斯莫拉 ||| 职业 ||| 运动员 足球  
question:39176369  question_id:巴莫斯 ||| 职业 ||| 运动员 足球  
question:41182628  question_id:拉莫斯 ||| 职业 ||| 足球运动员  
question:16616359  question_id:莫斯达 ||| 职业 ||| 运动员 足球  
question:25987545  question_id:阿斯莫 ||| 职业 ||| 运动员 足球  
question:23974383  question_id:莫伊斯 ||| 职业 ||| 运动员 足球  
question:25779348  question_id:苏莫斯 ||| 职业 |

KeyboardInterrupt: Interrupted by user

In [59]:
import json
data=json.loads(open("es_llm_table_tuple_top20_quchong_v3.json",encoding="utf8").read())

In [2]:
data[-1]

{'id': 'TECLS010',
 'question': '请仿照以下示例完成文本分类任务。示例：“李准基剪短发今日入伍将在杂志中公开服役心情新浪娱乐讯北京时间5月3日消息，据香港媒体报道，昨午(5月2日)，韩星李准基已在首尔剪发以预备今天入伍。李准基剪发的过程及服兵役的心情将在87月发行的杂志中公开，至于其父母也已在前日赶往首尔与他相聚道别。(俐俐)” 判断这段话属于体育、娱乐、科技、教育、财经中的哪个类别？ 答案：娱乐 请对以下题目进行文本分类：“台湾面板龙头友达青岛设厂本报讯(记者田丛)昨日，从台湾面板巨头友达光电传出消息，该公司旗下的背光模组工厂达运精密将投资379.4万美元，在青岛设立工厂。去年10月，海尔董事长张瑞敏应邀赴台，与友达董事长李?耀进行接触。这次将模组工厂设在青岛，业界猜测友达与海尔的合作将进入实质性阶段。”判断这段话属于体育、娱乐、科技、教育、财经中的哪个类别？ 答案：',
 'attribute': ['科技']}

In [6]:
from tqdm import tqdm
count=0
for unit in data[:-13]:
    temp_attr=[]
    for attr in unit["attribute"]:
        if "没有找到" not in attr:
            ss=attr.split("|||")
            if len(ss)>2:
                result = search_name.searchAnswer("|||".join(ss[:-1]))[0][2]
                if result!=attr:
                    count+=1
                    print(result,"===",attr)

王小诗 ||| 青年队 ||| 武汉光谷 === 王小诗 ||| 青年队 ||| 武汉光谷；王小诗 ||| 身高 ||| 1.88m；王小诗 ||| 位置 ||| 后卫、前锋；王小诗 ||| 体重 ||| 80kg
Amy Klinprathum ||| 星座 ||| 双子座 === Amy Klinprathum ||| 星座 ||| 双子座；Amy Klinprathum ||| 配偶 ||| Cee<Siwat Chotechaicharin
Amy Klinprathum ||| 身高 ||| 163 cm. === Amy Klinprathum ||| 出生地 ||| 美国；Amy Klinprathum ||| 身高 ||| 163 cm.
高桑大二朗 ||| 个人资料 ||| 高桑大二朗 === 高桑大二朗 ||| 个人资料 ||| 高桑大二朗；高桑大二朗 ||| 职业俱乐部* ||| 出场
哈塞克 ||| 人口（2007年） ||| 81,809 === 哈塞克 ||| 人口(2007年) ||| 81,809
芦芳生 ||| 体重 ||| 65kg === 芦芳生 ||| 中文名 ||| 芦芳生；芦芳生 ||| 身高 ||| 181cm；芦芳生 ||| 体重 ||| 65kg；芦芳生 ||| 血型 ||| O型
干毛毛 ||| 姐姐 ||| 干露露 === 干毛毛 ||| 籍贯 ||| 河南商城鲢鱼山人；干毛毛 ||| 职业 ||| 娱乐圈演员；干毛毛 ||| 出生日期 ||| 1989年2月21号；干毛毛 ||| 代表作品 ||| MV《爱你》；干毛毛 ||| 姐姐 ||| 干露露
刘少奇 ||| 副主席 ||| 董必武、宋庆龄（共同担任） === 刘少奇 ||| 副主席 ||| 董必武、宋庆龄(共同担任)
德马雷·卡罗尔 ||| 体重 ||| 96.2公斤/212磅 === 德马雷·卡罗尔 ||| 身高 ||| 2.03米/6英尺8英寸；德马雷·卡罗尔 ||| 体重 ||| 96.2公斤/212磅；德马雷·卡罗尔 ||| 职业生涯 ||| John Carroll in Birmingham, 阿拉巴马州
柴碧云 ||| 星座 ||| 巨蟹座 === 柴碧云 ||| 体重 ||| 41kg；柴碧云 ||| 爱好 ||| 拍摄广告；柴碧云 ||

熊倪 ||| 退休年份 ||| 1997年 → 1998年5月复出 2001年 === 熊倪 ||| 退役年份 ||| 1997年 → 1998年5月复出 2001年；熊倪 ||| 入选国家队 ||| 1986年1月；熊倪 ||| 主要奖项 ||| 1996年第26届奥运会三米跳板冠军 2000年奥运会男子3米板双人金牌 2000年奥运会蝉联个人三米板冠军；熊倪 ||| 身高 ||| 166cm；熊倪 ||| 籍贯 ||| 湖南长沙；熊倪 ||| 比赛项目 ||| 10米跳台→3米板
莱恩 ||| 施法距离 ||| 500 === 莱恩 ||| 国籍 ||| 艾泽拉斯王国；莱恩 ||| 施法距离 ||| 500；莱恩 ||| 阵营 ||| 联盟
袁树珊 ||| 去世年月 ||| 1968年11月8日 === 袁树珊 ||| 籍贯 ||| 扬州南乡；袁树珊 ||| 出生年月 ||| 1881年8月9日；袁树珊 ||| 去世年月 ||| 1968年11月8日；袁树珊 ||| 民族 ||| 汉
公遵法亲王 ||| 姓 ||| （无） === 公遵法亲王 ||| 姓 ||| (无)
金泰亨(Click-B成员) ||| 职业 ||| Click-B成员 === CLICK-B ||| 成立时间 ||| 1999年；CLICK-B ||| 成员 ||| 禹然皙，金泰亨，吴钟赫，金相赫，河贤坤，刘昊奭，卢敏赫；CLICK-B ||| 音乐专辑 ||| 《Click-B》 《Challenge》 《百战无败》 《Cowboy》 《The Best Of Click-B》
王光娜 ||| 婚姻 ||| 已婚（丈夫是韩国高尔夫球选手郑胜宇，并于2009年4月21日喜得贵子） === 王光娜 ||| 婚姻 ||| 已婚(丈夫是韩国高尔夫球选手郑胜宇，并于2009年4月21日喜得贵子)
韩松洞 ||| 占地面积 ||| 全长5.5英里（约8880米） === 韩松洞 ||| 占地面积 ||| 全长5.5英里(约8880米)
冷血动物(乐队) ||| 时间 ||| 1994 === 冷血动物乐队 ||| 唱片公司 ||| 骏驎唱片；冷血动物乐队 ||| 成立时间 ||| 1994年；冷血动物乐队 ||| 成员 ||| 谢天笑；李明；梁旭；冷血动物乐队 ||| 曲目 ||| 9首；冷血动物乐

蕾哈娜 ||| rihannanow.com ||| rihannanow.com === 蕾哈娜 ||| 网站 ||| www.rihannanow.com ||| rihannanow.com；蕾哈娜 ||| 出生 ||| Robyn Rihanna Fenty 1988年2月20日(26岁)[1] 巴巴多斯圣迈克尔区；蕾哈娜 ||| 活跃时期 ||| 2003至今；蕾哈娜 ||| 歌曲出处 ||| 第一首歌曲；蕾哈娜 ||| 互联网电影数据库(IMDb)信息 ||| 01060526RihannaNow.com(2005年)
麦家琪 ||| 语言 ||| 粤语，英语 === 麦家琪 ||| 语言 ||| 粤语，英语；麦家琪 ||| 籍贯 ||| 广东东莞；麦家琪 ||| 配偶 ||| 王国豪；麦家琪 ||| 英文名 ||| Teresa MIL；麦家琪 ||| 出生年月 ||| 1975年8月1日；麦家琪 ||| 教育程度 ||| 中五；麦家琪 ||| 活跃年代 ||| 1993-至今；麦家琪 ||| 中文名 ||| 麦家琪
白展堂 ||| 门派 ||| 葵花派 === 白展堂 ||| 体重 ||| 140斤；白展堂 ||| 门派 ||| 葵花派；白展堂 ||| 年龄 ||| 25岁；白展堂 ||| 武功 ||| 葵花点(解)穴手、还阳掌、轻功；白展堂 ||| 母亲 ||| 白三娘(白翠萍)；白展堂 ||| 妻子 ||| 佟湘玉；白展堂 ||| 儿子 ||| 白敬琪；白展堂 ||| 别名 ||| 白展堂；白展堂 ||| 姓名 ||| 白展堂
李乐伦 ||| 朝圣地（英语：Shrine） ||| 菲律宾马尼拉市岷伦洛区岷伦洛教堂 === 李乐伦 ||| 朝圣地(英语：Shrine) ||| 菲律宾马尼拉市岷伦洛区岷伦洛教堂
韩伍 ||| 籍贯 ||| 浙江杭州 === 韩伍 ||| 籍贯 ||| 浙江杭州；韩伍 ||| 出生年月 ||| 1936年
金城武 ||| 婚姻状况 ||| 未婚 === 金城武 ||| 出生日期 ||| 1973年(癸丑年)10月11日；金城武 ||| 信仰 ||| 佛教密宗；金城武 ||| 曾就读学校 ||| Taipei American School；金城武 ||| 婚姻状况 ||| 未婚
阿部浩之 ||| 出生日期 ||| 19

弗朗西斯科·法布雷加斯 ||| 专业特点 ||| 组织型中场 === 弗朗西斯科·法布雷加斯 ||| 所属运动队 ||| 切尔西足球俱乐部；弗朗西斯科·法布雷加斯 ||| 专业特点 ||| 组织型中场；弗朗西斯科·法布雷加斯 ||| 专业特点 ||| 组织型中场
熔岩巨兽 ||| 价格美服 ||| 975点券3150金币 === 熔岩巨兽 ||| 登场作品 ||| 《英雄联盟》；熔岩巨兽 ||| 价格美服 ||| 975点券3150金币；熔岩巨兽 ||| 类型 ||| 坦克、辅助、法师；熔岩巨兽 ||| 国服价格 ||| 1000点券1350金币
顾宝明 ||| 新浪微博 ||| http://t.sina.com.cn/kupaoming === 顾宝明 ||| 新浪微博 ||| http://t.sina.com.cn/kupaoming；顾宝明 ||| 出道地点 ||| 台湾；顾宝明 ||| 体重 ||| 67kg；顾宝明 ||| 身高 ||| 1.70m
徐丛林 ||| 普通话等级 ||| 普通话一级甲等认证 === 毛林林 ||| 普通话 ||| 北京普通话
许娜京 ||| 毕业院校 ||| 首尔艺术职业学校表演艺术学院 === 许娜京 ||| 外文名 ||| 하나경；许娜京 ||| 毕业院校 ||| 首尔艺术职业学校表演艺术学院；许娜京 ||| 体重 ||| 49kg；许娜京 ||| 身高 ||| 168cm
宋祖儿 ||| 表妹 ||| 唐怡然（然然）、唐怡佳（佳佳） === 宋祖儿 ||| 表妹 ||| 唐怡然(然然)、唐怡佳(佳佳)
马歇尔·D·汀奇 ||| 港澳译名 ||| 馬沙路·D·狄切（天下） === 马歇尔·D·汀奇 ||| 港澳译名 ||| 馬沙路·D·狄切(天下)
刘仁恩 ||| 本名 ||| 刘仁恩 === 刘仁恩 ||| 本名 ||| 刘仁恩；刘仁恩 ||| 出生地 ||| 不详；刘仁恩 ||| 去世时间 ||| 不详；刘仁恩 ||| 别名 ||| 刘仁恩；刘仁恩 ||| 主要成就 ||| 隋朝大将
沙加 ||| 日语声优 ||| 三矢雄二（初代）真殿光昭（LS） === 沙加 ||| 日语声优 ||| 三矢雄二(初代)真殿光昭(LS)
赵川 ||| 星座 ||| 金牛座 === 赵川 ||| 出生年月 ||| 1972年；赵川 

陈云良 ||| 出生年月 ||| 1965年6月 === 陈云良 ||| 出生年月 ||| 1965年6月；陈云良 ||| 出生地 ||| 湖南省双峰县
申世京 ||| 血 型 ||| B型 === 申世京 ||| 特长 ||| 钢琴、吉他；申世京 ||| 经纪公司 ||| Namooactors；申世京 ||| 爱好 ||| 听音乐 旅行；申世京 ||| 血型 ||| B型；申世京 ||| 英文名 ||| Shin Se Kyung；申世京 ||| 身高 ||| 166厘米；申世京 ||| 体重 ||| 48kg
朱由崧 ||| 父亲 ||| 恭宗孝皇帝 === 朱由崧 ||| 母亲 ||| 孝诚恭皇后；朱由崧 ||| 父亲 ||| 恭宗孝皇帝；朱由崧 ||| 妻子 ||| 孝哲简皇后
4月12日 ||| 后一天 ||| 4月13日 === 4月12日 ||| 含义 ||| 第102天；4月12日 ||| 出生名人 ||| 廖仲恺；4月12日 ||| 所属星座 ||| 白羊座；4月12日 ||| 逝世名人 ||| 富兰克林·罗斯福；4月12日 ||| 前后一天 ||| 4月13日；4月12日 ||| 含义 ||| 第102天；4月12日 ||| 纪念日 ||| 世界航天日；4月12日 ||| 后一天 ||| 4月13日；4月12日 ||| 逝世名人 ||| 富兰克林·罗斯福；4月12日 ||| 别名 ||| 4月12日
纪姿含 ||| 喜欢的食物 ||| 冰淇淋、棒棒糖、蛋糕 === 纪姿含 ||| 喜欢的食物 ||| 冰淇淋、棒棒糖、蛋糕；纪姿含 ||| 出生日期 ||| 2007年10月9日[八月廿九][5]；纪姿含 ||| 身高 ||| 121cm(成长中)；纪姿含 ||| 体重 ||| 17公斤
南希·韦克 ||| 1945年的南希·韦克 ||| 1945年的南希·韦克 === 南希·韦克 ||| 出生年月 ||| 1912年8月13日；南希·韦克 ||| 去世年月 ||| 2011年8月7日；南希·韦克 ||| 职业 ||| 其他 间谍；南希·韦克 ||| 1945年的南希·韦克 ||| 1945年的南希·韦克
恩维尔·霍查 ||| 出生年月 ||| 1908年10月16日 === 恩维尔·霍查 ||| 逝世年月 ||| 1985年4月11日；恩维尔·霍

闫学晶 ||| 奖项 ||| “纪念成兆才诞辰120周年评剧新剧目汇演”配角一等奖 1994年 《多彩的梦》 文华奖 1999年 《胡知县断案》 吉林省二人转推广会一等奖 2000年 《皇帝梦》 === 闫学晶 ||| 现住地 ||| 北京朝阳区；闫学晶 ||| 奖项 ||| “纪念成兆才诞辰120周年评剧新剧目汇演”配角一等奖 1994年 《多彩的梦》 文华奖 1999年 《胡知县断案》 吉林省二人转推广会一等奖 2000年 《皇帝梦》；闫学晶 ||| 其他成就 ||| 现代评剧《多彩的梦》获配角一等奖， 吉林省二人转推广会一等奖， 春节晚会相声剧《公交谐奏曲》一等奖；闫学晶 ||| 音乐作品 ||| 《永远伴随你一生》, 《关东情》, 《老百姓的事情大于天》, 《打是亲来骂是爱》；闫学晶 ||| 毕业院校 ||| 吉林省戏曲学校；闫学晶 ||| 出身地 ||| 吉林省辽源市；闫学晶 ||| 奖项 ||| “纪念成兆才诞辰120周年评剧新剧目汇演”配角一等奖 1994年 《多彩的梦》 文华奖 1999年 《胡知县断案》 吉林省二人转推广会一等奖 2000年 《皇帝梦》
法斯琪 ||| 登场作品 ||| 魔兽 === 法斯琪 ||| 登场作品 ||| 魔兽；法斯琪 ||| 种族 ||| 娜迦族；法斯琪 ||| 能力(招术) ||| 叉状闪电、冰霜之箭、法力护盾和旋风四个技能；法斯琪 ||| 所属作品 ||| 《魔兽争霸》；法斯琪 ||| 作者 ||| 暴雪公司；法斯琪 ||| 初登场时间 ||| 2003年开始；法斯琪 ||| 作品 ||| 《WARCRAFT3》
亚伦·斯沃茨 ||| 逝世 ||| 2013年1月11日（26岁）[1] 纽约布鲁克林区 === 亚伦·斯沃茨 ||| 逝世 ||| 2013年1月11日(26岁)[1] 纽约布鲁克林区
高贤廷 ||| 出生年月 ||| 1971年3月2日 === 高贤廷 ||| 职业 ||| 演员；高贤廷 ||| 出生年月 ||| 1971年3月2日；高贤廷 ||| 主要奖项 ||| 1989年韩国小姐选美赛亚军、 28届百想艺术大赏女子新人演技赏、 2005 SBS演技大赏10大明星赏、 2009 MBC演技大赏 最高大赏、 2010第46届百想艺术大赏TV大赏
风清扬 ||| 派别 ||| 华山派剑宗 === 风清

少羽 ||| 能力（招术） ||| 破阵, 横扫千军, 万人敌 === 少羽 ||| 能力(招术) ||| 破阵， 横扫千军， 万人敌；少羽 ||| 所属公司 ||| 玄机科技
桑兰西 ||| 政党 ||| 桑兰西党 === 桑兰西 ||| 中文名 ||| 桑兰西；桑兰西 ||| 政党 ||| 桑兰西党；桑兰西 ||| 外文名 ||| Sam Rainsy；桑兰西 ||| 主要成就 ||| 回国参加大选活动
车太贤 ||| 最喜欢的食物 ||| 快餐 === 车太贤 ||| 最喜欢的食物 ||| 快餐；车太贤 ||| 最喜欢的部位 ||| 额头；车太贤 ||| 出道 ||| 1995年KBS超级演艺班第一期；车太贤 ||| 最喜欢的食物 ||| 快餐；车太贤 ||| 毕业院校 ||| 汉城艺术大学，京畿大学；车太贤 ||| 经纪公司 |||Sidus HQ；车太贤 ||| 身高 ||| 175厘米
丁得孙 ||| 兵器 ||| 飞叉 === 丁得孙 ||| 人物角色 ||| 张清手下副将；丁得孙 ||| 人物武器 ||| 白虎叉、虎牙飞叉；丁得孙 ||| 梁山排名 ||| 79；丁得孙 ||| 兵器 ||| 飞叉；丁得孙 ||| 别名 ||| 丁得孙；丁得孙 ||| 出身 ||| 东昌府副将
眼镜蛇乐队 ||| 音乐类型 ||| 摇滚 === 眼镜蛇乐队 ||| 音乐类型 ||| 摇滚；眼镜蛇乐队 ||| 唱片公司 ||| 新蜂音乐；眼镜蛇乐队 ||| 成立时间 ||| 1989年；眼镜蛇乐队 ||| 成员 ||| 王晓芳 ||| 鼓手；王晓芳 ||| 成员 ||| 王晓芳 ||| 主唱；王晓芳 ||| 音乐类型 ||| 摇滚；王晓芳 ||| 唱片公司 ||| 新蜂音乐
中国娃娃 ||| 国籍 ||| 中国 === 中国娃娃 ||| 英文名 ||| China Dolls；中国娃娃 ||| 经纪公司 ||| 台湾大声音乐国际股份有限公司；中国娃娃 ||| 首张泰语专辑 ||| 《Muay Nee Kah》；中国娃娃 ||| 国籍 ||| 中国；中国娃娃 ||| 职业 ||| 歌手；中国娃娃 ||| 中文名 ||| 中国娃娃
坂本辰马 ||| 特征 ||| 毛球头 太阳镜 === 坂本辰马 ||| 性格 ||| 乐天派；坂本辰马 ||| 特征 ||| 毛球头 太阳镜；

魏成 ||| 魏成 ||| 季成、魏成子 === 魏成 ||| 出生地 ||| 江苏无锡；魏成 ||| 名字来源于 ||| 季成、魏成子；魏成 ||| 所处时代 ||| 周朝战国时代；魏成 ||| 国家 ||| 魏国；魏成 ||| 官位 ||| 魏相
赫南格 ||| 惯用脚 ||| 右脚 === 赫南格 ||| 所属运动队 ||| 萨尔普斯堡；赫南格 ||| 惯用脚 ||| 右脚；赫南格 ||| 场上位置 ||| 前卫；赫南格 ||| 球衣号码 ||| 21号
大蛇丸 ||| 师傅 ||| 猿飞日斩（三代火影） === 大蛇丸 ||| 师傅 ||| 猿飞日斩(三代火影)
申世景 ||| 语言 ||| 韩语、英语 === 申世景 ||| 出生日期 ||| 1990年7月29日；申世景 ||| 经纪公司 ||| Namooactors；申世景 ||| 语言 ||| 韩语、英语；申世景 ||| 教育程度 ||| 中央大学戏剧电影学系(休学)；申世景 ||| 血型 ||| B型；申世景 ||| 身高 ||| 167cm；申世景 ||| 职业 ||| 演员；申世景 ||| 体重 ||| 43kg
童安格 ||| 代表作品 ||| 《其实你不懂我的心》《明天你是否依然爱我》《把根留住》 === 童安格 ||| 签约公司 ||| 东盛时代国际传媒；童安格 ||| 血型 ||| A；童安格 ||| 生肖 ||| 猪；童安格 ||| 出生 ||| 1959年7月26日(55岁)；童安格 ||| 最出名的作品 ||| 耶利亚女郎、把根留住、想你；童安格 ||| 身高 ||| 176cm；童安格 ||| 代表作品 ||| 《其实你不懂我的心》《明天你是否依然爱我》《把根留住》
陈柏宇 ||| 女朋友 ||| 符晓薇（香港模特儿） === 陈柏宇 ||| 女朋友 ||| 符晓薇(香港模特儿)
后土御门天皇 ||| 陵所 ||| 深草北陵 般舟院陵 === 后土御门天皇 ||| 皇居 ||| 京都御所；后土御门天皇 ||| 陵所 ||| 深草北陵 般舟院陵；后土御门天皇 ||| 崩御 ||| 1500年10月21日；后土御门天皇 ||| 前任 ||| 后花园天皇；后土御门天皇 ||| 陵墓 ||| 深草北陵 般舟院陵；后土御门天皇 ||| 年号 ||| 宽正 文正 应仁 文明 长享 延德 明

陈乔森 ||| 去世年月 ||| 1905年 === 陈乔森 ||| 出生年月 ||| 1835年；陈乔森 ||| 去世年月 ||| 1905年；陈乔森 ||| 籍贯 ||| 遂溪县
卡里卡里 ||| 身高 ||| 187.0CM === 卡里 ||| 身高 ||| 183厘米；卡里卡里 ||| 外文名 ||| CaryCary；卡里 ||| 身高 ||| 184厘米
任容萱 ||| 就读国中 ||| 台北市立兰雅国民中学 === 任容萱 ||| 兴趣 ||| 跳舞、唱歌、弹钢琴；任容萱 ||| 教育程度 ||| 芝山国小毕业 兰雅国中毕业 中正高中毕业 国立台湾师范大学人类发展与家庭学系家政与家庭生活教育组学士；任容萱 ||| 出生 ||| 1988年11月22日(26岁) 中华民国台北市；任容萱 ||| 就读小学 ||| 台北市立芝山国小；任容萱 ||| 就读高中 ||| 台北市立中正高级中学；任容萱 ||| 就读国中 ||| 台北市立兰雅国民中学
何音 ||| 出生年月 ||| 3月16日 === 何音 ||| 配偶 ||| 黄志忠(1997年-2011年)；何音 ||| 身高 ||| 165 cm；何音 ||| 出生日期 ||| 1967年3月16日；何音 ||| 毕业学校 ||| 四川大学；何音 ||| 血型 ||| A型
阎红彦 ||| 去世年月 ||| 1967年1月8日 === 阎红彦 ||| 出生年月 ||| 1909年10月27日；阎红彦 ||| 籍贯 ||| 陕西省安定(今子长)县；阎红彦 ||| 去世年月 ||| 1967年1月8日
A-RISE ||| 就读院校 ||| UTX === A仔 ||| 就读的学校 ||| 狗刨中学【正在就读】
1276年 ||| 1276年逝世 ||| 海梅一世，阿德里安五世等 === 1276年 ||| 1276年逝世 ||| 海梅一世，阿德里安五世等；1276年 ||| 1276年出生 ||| 元泰定帝，久明亲王；1276年 ||| 世纪 ||| 12世纪 | 13世纪 | 14世纪；1276年 ||| 中文名 ||| 1276年；1276年 ||| 归属 ||| 英仙座；1276年 ||| 主要事件 ||| 元兵破潭州；1276年 ||| 年份 ||| 丙子年(鼠年)
1276年 ||| 1276年逝世

In [7]:
count

466

In [60]:
temp_data=json.loads(open("es_llm_top20_test_quchong_v3.json",encoding="utf8").read())


In [11]:
temp_data[0]

{'id': 'TE000138',
 'question': '恩金·巴伊塔尔作为土耳其运动员，在球队中担负的职责是什么',
 'attribute': ['恩金·巴伊塔尔 ||| 国籍 ||| 土耳其/德国'],
 'tuple_predict': ['恩金·巴伊塔尔场上职责'],
 'top20': ['恩金·巴伊塔尔 ||| 司职 ||| 中场、边锋',
  '恩金·巴伊塔尔 ||| 别名 ||| 恩金·巴伊塔尔',
  '恩金·巴伊塔尔 ||| 国籍 ||| 土耳其/德国',
  '恩金·巴伊塔尔 ||| 体重 ||| 69.0 KG',
  '恩金·巴伊塔尔 ||| 身高 ||| 180.0 CM',
  '恩金·巴伊塔尔 ||| 姓名 ||| 恩金·巴伊塔尔',
  '恩金·巴伊塔尔 ||| 球衣 ||| 50号',
  '恩金·巴伊塔尔 ||| 出生地 ||| 居特斯洛（德国）',
  '恩金·巴伊塔尔 ||| 外文名 ||| Engin Baytar',
  '恩金·巴伊塔尔 ||| 出生日期 ||| 1982-03-31',
  '恩金·巴伊塔尔 ||| 运动项目 ||| 足球',
  '恩金·巴伊塔尔 ||| 重要事件 ||| 代表土耳其出场3次 曾效力于特拉布宗体育',
  '恩金·巴伊塔尔 ||| 所属运动队 ||| 加拉塔萨雷足球俱乐部',
  '塔拉·琳恩·巴尔 ||| 职业 ||| 演员',
  '伊塔诺·巴尔博 ||| 职业 ||| 法西斯头目',
  '伊塔马尔·巴蒂斯塔 ||| 职业 ||| 足球运动员',
  '莫伊塔巴·贾巴里 ||| 场上位置 ||| 前卫',
  '巴尔(塔恩省) ||| 巴尔 Barre ||| 法国',
  '伊恩·詹姆斯·巴尔 ||| 职业 ||| 游泳运动员',
  '恩加尔塔·托姆巴巴耶 ||| 职业 ||| 政治 总统',
  '诺伊恩塔尔 ||| 州 ||| 黑森州',
  '肖恩·巴蒂尔 ||| 场上位置 ||| 小前锋',
  '伊恩·巴肯 ||| 职业 ||| 教练',
  '巴尔(塔恩省) ||| 区 ||| 卡斯特尔区 (Castres)',
  '巴尔(塔恩省) ||| 县 ||| 韦布勒河畔米拉县 (Murat-sur-Vèbre)',
  '巴尔(塔恩省) ||| 省 |||

In [61]:
count=0
for unit,unit2 in tqdm(zip(data[:-13],temp_data[:-13])):
    temp_attr=[]
    for attr in unit["attribute"]:
        if attr:
            if "没有找到" not in attr:
                if "；" in attr:
                    attr=attr.split("；")
                    for a in attr:
                        a=a.split("|||")
                        if len(a)>2:
                            result = search_name.searchAnswer("|||".join(a[:-1]))[0][2]
                            temp_attr.append(result)
                else:
                    attr=attr.split("|||")
                    if len(attr)>2:
                        result = search_name.searchAnswer("|||".join(attr[:-1]))[0][2]
                        temp_attr.append(result)
            else:
                temp_attr.append(unit2["top20"])
    unit["attribute"] = temp_attr+[k for k in unit2["top20"] if k not in temp_attr]

2000it [00:52, 38.38it/s]


In [53]:
data[308]["attribute"]

['雷吉·布洛克 ||| 所属运动队 ||| 菲尼克斯太阳队',
 '雷吉·布洛克 ||| 球衣号码 ||| --号',
 '雷吉·布洛克 ||| 运动项目 ||| 篮球',
 '雷吉·布洛克 ||| 出生日期 ||| 1991年3月16日',
 '雷吉·布洛克 ||| 身高 ||| 2.01米/6英尺7英寸',
 '雷吉·布洛克 ||| 体重 ||| 91公斤/200磅',
 '雷吉·布洛克 ||| 毕业院校 ||| 北卡罗莱纳大学',
 '雷吉·布洛克 ||| NBA选秀 ||| 2013年第一轮第25位被快船队选中',
 '雷吉·布洛克 ||| 职业 ||| 篮球',
 '雷吉·布洛克 ||| 别名 ||| 雷吉·布洛克',
 '雷吉·布洛克 ||| 国籍 ||| 美国',
 '雷吉·布洛克 ||| 性别 ||| 男',
 '雷吉·布洛克 ||| 英文名 ||| Reggie Bullock',
 '雷吉·布洛克 ||| 中文名 ||| 雷吉·布洛克',
 '雷吉·布洛克 ||| 出生地 ||| 美国马里兰州巴尔的摩',
 '雷吉·布洛克 ||| 外文名 ||| Reggie Bullock',
 '雷吉·布洛克 ||| 出生年月 ||| 1991年3月16日',
 '雷吉洛克 ||| 属 性 ||| 岩石',
 '雷吉洛克 ||| 属性 ||| 岩石',
 '穆奇·布雷洛克 ||| NBA选秀 ||| 1989年 / 第1轮 / 第12顺位',
 '威尔·布雷洛克 ||| NBA选秀 ||| 2006年 / 第2轮 / 第60顺位',
 '穆奇·布雷洛克 ||| 球队 ||| 金州勇士队',
 '雷吉洛克 ||| 所属作品 ||| 《口袋妖怪》',
 '雷吉洛克 ||| 所属公司 ||| 日本任天堂公司',
 '布雷吉 ||| 所属运动队 ||| 伯尔尼小伙子',
 '雷吉洛克 ||| 别名 ||| 雷吉洛克',
 '雷吉洛克 ||| 体重 ||| 230kg',
 '雷吉洛克 ||| 姓名 ||| 雷吉洛克',
 '雷吉洛克 ||| 配音 ||| 小西克幸（TV） 宫下荣治（M08）',
 '雷吉洛克 ||| 性别 ||| 无性别']

In [16]:
set(list([1,2,3,1,5,2]))

{1, 2, 3, 5}

In [24]:
unit

{'id': 'TE000870',
 'question': '谁是周侗的义子？',
 'attribute': ['侗寨 ||| 释义 ||| 侗族村落',
  '周侗 ||| 外号 ||| 陕西大侠、铁臂膀',
  '周子义 ||| 性别 ||| 男',
  '侗族简史 ||| 意义 ||| 使人们更加了解侗族',
  '周侗 ||| 国籍 ||| 北宋',
  '周侗(宋代名将岳飞的启蒙老师) ||| 义子 ||| 岳飞',
  '周侗 ||| 本名 ||| 周侗',
  '周侗 ||| 去世时间 ||| 1119年',
  '周侗 ||| 不记名弟子 ||| 武松',
  '周子义 ||| 籍贯 ||| 广东开平',
  '周侗 ||| 义子 ||| 岳飞',
  '周侗 ||| 别称 ||| 周光祖',
  '周侗 ||| 逝世日期 ||| 1119年',
  '周侗 ||| 出生时间 ||| 约1040年',
  '周子义 ||| 中文名 ||| 周子义',
  '周侗 ||| 徒弟 ||| 卢俊义、林冲、史文恭',
  '周侗 ||| 中文名 ||| 周侗',
  '周侗 ||| 主要成就 ||| 传授武艺、开创鹰爪翻子门',
  '侗 ||| 释义 ||| 侗 侗族。中国少数民族名 侗 诚实忠厚 侗〈形〉 tong垌tǒng 1.缶垌。 2.姓。宋代有垌夫。见《正字通．土部',
  '周侗 ||| 职业 ||| 武术家、爱国人士',
  '周侗 ||| 所处时代 ||| 北宋末年',
  '周侗 ||| 出生地 ||| 陕西',
  '周子义 ||| 民族 ||| 汉族',
  '周侗 ||| 别名 ||| 周侗',
  '周侗 ||| 民族 ||| 汉族',
  '侗乡鼓楼 ||| 意义 ||| 侗乡的重要标志之一',
  '周侗 ||| 出生日期 ||| 约1040年',
  '周子义 ||| 别名 ||| 周子义',
  '周子义 ||| 国籍 ||| 中国',
  '周侗 ||| 代表作品 ||| 关中红拳']}

In [62]:
submission = open("es_llm_table_tuple_top20_quchong_post_process_v3_2.json", "w", encoding="utf8")

submission.write(json.dumps(data, ensure_ascii=False))

2498707

In [45]:
data[2]

{'id': 'TE001070',
 'question': '哪些官职在松平赖桓的身上',
 'attribute': ['松平赖桓 ||| 朝廷官位 ||| 从四位上、讃岐守、侍从']}

In [52]:
st='霸州市新利钢铁有限公司 ||| 别名 ||| 霸州市新利钢铁有限公司'
str(set(st))

"{'钢', '限', '利', '|', ' ', '新', '司', '州', '名', '市', '霸', '别', '公', '有', '铁'}"

In [53]:
import json
data=json.loads(open("es_llm_top20_test_quchong.json",encoding="utf8").read())

In [61]:
final_result=[]
for unit in data[:-13]:
    temp_dic={
        "id":unit["id"],
        "question":unit["question"],
        "attribute":unit["top20"]
    }
    final_result.append(temp_dic)

In [63]:
for unit in data[-13:]:
    temp_dic={
        "id":unit["id"],
        "question":unit["question"],
        "attribute":unit["attribute"]
    }
    final_result.append(temp_dic)

In [59]:
final_result[3]

{'id': 'TE000550',
 'question': '什么是杰基·乔伊纳一克西获得的单人最好成就？',
 'attribute': ['杰基·乔伊纳一克西 ||| 个人最好成绩 ||| 女子七项全能7291分',
  '杰基·乔伊纳一克西 ||| 别名 ||| 杰基·乔伊纳一克西',
  '杰基·乔伊纳一克西 ||| 国籍 ||| 美国',
  '杰基·乔伊纳一克西 ||| 中文名 ||| 杰基·乔伊纳·克西',
  '杰基·乔伊纳一克西 ||| 出生地 ||| 伊利诺伊州',
  '杰基·乔伊纳一克西 ||| 外文名 ||| JACKIEJOYNER－KERSEE',
  '杰基·乔伊纳一克西 ||| 出生日期 ||| 1962年3月3日',
  '杰基·乔伊纳一克西 ||| 运动项目 ||| 七项全能',
  '杰基·乔伊纳一克西 ||| 主要奖项 ||| 第24届奥运会跳远金牌 第24届奥运会七项全能金牌',
  '杰基·乔伊纳一克西 ||| 重要事件 ||| 首位突破女子七项全能“7000分”',
  '乔纳森·杰克逊 ||| 主要成就 ||| 艾美奖',
  '乔伊纳·克西 ||| 性别 ||| 女',
  '乔伊纳·克西 ||| 国籍 ||| 美国',
  '乔伊纳·克西 ||| 别名 ||| 乔伊纳·克西',
  '乔伊纳·克西 ||| 职业 ||| 田径',
  '乔伊纳·克西 ||| 英文名 ||| Joyner Kersee',
  '乔伊纳·克西 ||| 出生地 ||| 美国的伊利诺伊州',
  '乔伊纳·克西 ||| 中文名 ||| 乔伊纳•克西',
  '乔纳森·伊夫 ||| 主要成就 ||| 参与设计苹果产品',
  '乔纳森·盖伊 ||| 主要成就 ||| Flash 最初的设计者']}

In [64]:
submission = open("es_llm_tuple_top20_quchong.json", "w", encoding="utf8")

submission.write(json.dumps(final_result, ensure_ascii=False))

1386413

In [65]:
import json
data=json.loads(open("es_top20_train.json",encoding="utf8").read())

In [73]:
sums,maxs=[],0
for unit in data:
    length=len(unit["attribute"])
    if length>maxs:
        maxs=length

In [74]:
maxs

27